In [ ]:
# coding: utf-8
import os
import numpy as np
import sys
import pickle
import warnings
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

# Dataset.pyの格納先を環境変数に追加する
sys.path.append("../src/utils")
sys.path.append("../src")
from Dataset import Dataset
from FaceDetect import FaceCrop

# 転移学習用
from keras import optimizers
from keras.utils.np_utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Flatten, Input, Activation
from keras.models import Model, Sequential
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint

from keras.models import load_model

%matplotlib inline

In [ ]:
# モデルの読み込み
model_dir = "../models/VGG16_finetuning"
model_path = "{}/VGG16.h5".format(model_dir)

model = load_model(model_path, compile=False)
print("モデルの概要")
model.summary()

In [ ]:
# 学習済み重みのロード
weights_path = "{}/VGG16_93.h5".format(model_dir)
model.load_weights(weights_path)

In [ ]:
# データセットを定義
dataset = Dataset()
# pickleの格納先
pickle_path = "../data/dataset.pkl"
# pickleからデータセットを読み込む
dataset.read_pickle(pickle_path)

# 学習データとテストデータの割合
ratio = 0.2
# 学習データとテストデータに分ける
dataset.split_data(ratio)

dataset.y_train = to_categorical(dataset.y_train)
dataset.y_test = to_categorical(dataset.y_test)

In [ ]:
# 顔検出用
face_cascade_path = '../models/xml/haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(face_cascade_path)
demo_dir = "../data/demo"

In [ ]:
print("-"*50)
print("画像に写っている人が好みか判定するよ！")
print("写真のファイル名を入力してね: ", end="")
img_name = input()
print("")
print("")
img_path = "{}/{}".format(demo_dir, img_name)
try:
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
except:
    print("ファイル名を確認してください！")

print("↓", img_name)
plt.imshow(img)
plt.show()

print("顔を検出します...")
faces = face_cascade.detectMultiScale(gray, 1.3, 5)
detect_faces = []
if len(faces)==0:
    print("顔が検出されませんでした")
for i, (x,y,w,h) in enumerate(faces):
    detect_img = FaceCrop(img, ratio, x, y, w, h)
    detect_img = cv2.resize(detect_img, dsize=(100, 100))
    plt.imshow(detect_img)
    plt.show()
    detect_img = detect_img.reshape((1, 100, 100, 3))
    print("[好みじゃない確率, 好みの確率]")
    y = model.predict(detect_img)
    print(y)